In [1]:

from IPython.display import display

In [2]:
import ipywidgets as widgets
import tempfile
import os, shutil

In [3]:
import subprocess

In [4]:
tmpdir = tempfile.mkdtemp(suffix=None, prefix=None, dir=None)

In [5]:
ROOT=os.getcwd()

In [6]:
#default style
style = {'description_width': 'initial'}

#layout=widgets.Layout(width='50%')
layout=widgets.Layout()

In [7]:
# input files widget
sequence_file_upload_label = widgets.Label(value="Select sequence files (.fas)")
sequence_file_upload_widget = widgets.FileUpload(
                        accept='fas, fasta, fa',
                        multiple=True,
                        disabled=False,
                        description="Select sequence files (.fas)",
                        layout=widgets.Layout(width='50%'),
)

reference_strain_label = widgets.Label("Select the reference strain")
reference_strain_widget = widgets.Dropdown(
                            options=["Select the reference strain"],
                            value="Select the reference strain",
                            description="",
                            disabled=True,
                            layout=sequence_file_upload_widget.layout,
)

def on_sequence_file_upload_change(change):
    # Write uploded files to tmpdir
    
    for strain in change['new'].values():
        with open(os.path.join(tmpdir, strain['metadata']['name']), 'wb') as fp:
            fp.write(strain['content'])
            
    # Set options on reference_strain_widget
    reference_strain_widget.options=list(change['new'].keys())
    reference_strain_widget.disabled = False

sequence_file_upload_widget.observe(on_sequence_file_upload_change, names='value')

In [8]:
# parameter widgets
min_nmer_label = widgets.Label("Min nmer occurence", layout=widgets.Layout(width='30%'))
min_nmer_widget = widgets.IntSlider(value=55,
                                            min=0,
                                            max=100,
                                            description="",
                                            layout=widgets.Layout(width='50%'),
                                   )


nmer_length_label = widgets.Label("Nmer length", layout=widgets.Layout(width='30%'))
nmer_length_widget = widgets.IntSlider(value=21,
                                            min=0,
                                            max=100,
                                            description="",
                                            layout=widgets.Layout(width='50%'),
                                      )

In [9]:
query_rayt_upload_label=widgets.Label("Select query rayt sequence file (.faa)")
query_rayt_upload_widget = widgets.FileUpload(
    accept="faa",
    multiple=False,
    description="Select query rayt sequence file (.faa)",
    layout=widgets.Layout(width='50%'),
)

query_rayt_fname = ''
def on_query_rayt_upload_change(change):
    # Write uploded files to tmpdir
    
    blob = list(change['new'].values())[0]
    query_rayt_fname = blob['metadata']['name']
    content = blob['content']
    with open(os.path.join(tmpdir, query_rayt_fname), 'wb') as fp:
        fp.write(content)
            
query_rayt_upload_widget.observe(on_query_rayt_upload_change, names='value')

In [10]:
tree_file_label = widgets.Label("Name (path) of the tree file. Will be generated if not present.")
tree_file_widget = widgets.Text(value="",
                                         placeholder="Name of tree file. Will be generated if not present.",
                                         description="",
                                         disabled=False,
                                        layout=widgets.Layout(width='50%')
                               )

In [11]:
e_widget = widgets.FloatLogSlider(
    value=1.0e-90,
    base=10,
    min=-100, # max exponent of base
    max=0, # min exponent of base
    step=0.1, # exponent step
    description='e value cutoff'
)



In [12]:
def check_all_inputs():
    return all([True])

In [13]:
output_widget = widgets.Output()

In [14]:
run_button = widgets.Button(
    description='Run RepinPop analysis',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run the Repin Population analysis',
    icon='run',
    layout=widgets.Layout(width='50%')
)

In [15]:
def on_button_clicked(b):
    with output_widget:
        print("RepinPop starts")
        run_repin_pop()

In [16]:
run_button.on_click(on_button_clicked)

In [17]:
def run_repin_pop():
    oldwd = os.getcwd()
    #os.chdir(tempfile.gettempdir())
    os.chdir('/')
    command = ['java',
              '-jar',
              os.path.abspath(
                  os.path.join(ROOT,
                               '..',
                               'REPIN_ecology/REPIN_ecology/build/libs/REPIN_ecology.jar')),
              tmpdir,
               reference_strain_widget.value,
               '{0:d}'.format(min_nmer_widget.value),
               '{0:d}'.format(nmer_length_widget.value),
               os.path.join(tmpdir, list(query_rayt_upload_widget.value.values())[0]['metadata']['name']),
               os.path.join(tmpdir, tree_file_widget.value),
               '{0:e}'.format(e_widget.value)
              ]
    
    
    #print(command)
    
    with subprocess.Popen(command,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT) as proc:
        
        
        print(proc.stdout.read())
            

    
    os.chdir(oldwd)
    

In [18]:
gui = widgets.VBox(
    [
        widgets.HBox(
        [
            sequence_file_upload_widget,
            reference_strain_widget,
        ]
        ),
        widgets.HBox(
        [
            query_rayt_upload_widget,
            tree_file_widget
        ]
        ),
        widgets.HBox(
        [
            min_nmer_label,
            min_nmer_widget,
        ]
        ),
        widgets.HBox(
        [
            nmer_length_label,
            nmer_length_widget       
        ]),
        widgets.HBox(
        [
            e_widget,
            run_button,
        ]
        ),
        output_widget
    ],
    #layout=widgets.Layout(width='100%')
)

In [19]:
_ = display(gui)